In [1]:
import numpy as np
import pandas as pd

<h4>Transformacion de datasets plataformas de streaming</h4>

In [2]:
#Cargar los datasets
amazon = pd.read_csv('datasets/amazon_prime_titles.csv')
disney = pd.read_csv('datasets/disney_plus_titles.csv')
hulu = pd.read_csv('datasets/hulu_titles.csv')
netflix = pd.read_csv('datasets/netflix_titles.csv')

In [ ]:
#Agregar columna id con la inicial de cada plataforma
amazon.insert(0, 'id', 'a' + amazon['show_id'].astype('string'))
disney.insert(0, 'id', 'd' + disney['show_id'].astype('string'))
hulu.insert(0, 'id', 'h' + hulu['show_id'].astype('string'))
netflix.insert(0, 'id', 'n' + netflix['show_id'].astype('string'))


In [6]:
#Agregar 'G' en las filas faltantes del atributo rating
amazon['rating'] = amazon['rating'].fillna('G')
disney['rating'] = disney['rating'].fillna('G')
hulu['rating'] = hulu['rating'].fillna('G')
netflix['rating'] = netflix['rating'].fillna('G')


In [7]:
#Transformar fechas al formato yyy-mm-dd
amazon['date_added'] = pd.to_datetime(amazon['date_added'])
disney['date_added'] = pd.to_datetime(disney['date_added'])
hulu['date_added'] = pd.to_datetime(hulu['date_added'])
netflix['date_added'] = pd.to_datetime(netflix['date_added'])

In [8]:
#Dividir la columna duration en duration_int y duration_type
amazon[['duration_int', 'duration_type']] = amazon.duration.str.split(' ', expand=True)
disney[['duration_int', 'duration_type']] = disney.duration.str.split(' ', expand=True)
hulu[['duration_int', 'duration_type']] = hulu.duration.str.split(' ', expand=True)
netflix[['duration_int', 'duration_type']] = netflix.duration.str.split(' ', expand=True)

In [9]:
#Cambiar todo el texto a minusculas
listcolum = ['type','title','director','cast','country','listed_in','description','duration_type']
for item in listcolum:
    amazon[item] = amazon[item].str.lower()
for item in listcolum:
    disney[item] = disney[item].str.lower()
for item in listcolum:
    hulu[item] = hulu[item].astype('string') 
for item in listcolum:
    hulu[item] = hulu[item].str.lower()  
for item in listcolum:
    netflix[item] = netflix[item].str.lower()      

In [10]:
#Export csv
amazon.to_parquet('./transformed_datasets/amazon.parquet')
disney.to_parquet('./transformed_datasets/disney.parquet')
hulu.to_parquet('./transformed_datasets/hulu.parquet')
netflix.to_parquet('./transformed_datasets/netflix.parquet')

<h4>Transformacion de datasets ratings</h4>

In [3]:
data = pd.read_csv('ratings/ratings_u.csv')

In [ ]:
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')
data = data.drop(columns=['Unnamed: 0'])

In [6]:
data.to_parquet('transformed_datasets/ratings_unified.parquet')